In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [2]:
import kagglehub

path = kagglehub.dataset_download("paultimothymooney/poetry")

print("Path to dataset files:", path)

100%|██████████| 2.00M/2.00M [00:00<00:00, 74.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/paultimothymooney/poetry/versions/16


In [20]:
ls /root/.cache/kagglehub/datasets/paultimothymooney/poetry/versions/16/

adele.txt              dolly-parton.txt        ludacris.txt
al-green.txt           drake.txt               michael-jackson.txt
alicia-keys.txt        dr-seuss.txt            missy-elliott.txt
amy-winehouse.txt      eminem.txt              nickelback.txt
beatles.txt            janisjoplin.txt         nicki-minaj.txt
bieber.txt             jimi-hendrix.txt        nirvana.txt
bjork.txt              johnny-cash.txt         notorious-big.txt
blink-182.txt          joni-mitchell.txt       notorious_big.txt
bob-dylan.txt          kanye.txt               nursery_rhymes.txt
bob-marley.txt         kanye-west.txt          patti-smith.txt
britney-spears.txt     Kanye_West.txt          paul-simon.txt
bruce-springsteen.txt  lady-gaga.txt           prince.txt
bruno-mars.txt         leonard-cohen.txt       radiohead.txt
cake.txt               lil-wayne.txt           rihanna.txt
dickinson.txt          Lil_Wayne.txt           r-kelly.txt
disney.txt             lin-manuel-miranda.txt
dj-khaled.txt       

In [9]:
import os
from datasets import Dataset

# Assuming your Kaggle dataset is downloaded to a directory
dataset_path = "/root/.cache/kagglehub/datasets/paultimothymooney/poetry/versions/16"

text_data = []
for filename in os.listdir(dataset_path):
    if filename.endswith(".txt"):
        with open(os.path.join(dataset_path, filename), "r", encoding="utf-8") as f:
            text = f.read().strip()
            if text:  # Skip empty files
                text_data.append({"text": text + " <|endoftext|>"})

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(text_data)


In [10]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [13]:

tokenized_dataset = tokenized_dataset.map(
    lambda examples: {
        "input_ids": examples["input_ids"],
        "attention_mask": examples["attention_mask"],
        "labels": examples["input_ids"].copy()  # Labels = inputs for LM
    },
    batched=True
)


split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [16]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

model = GPT2LMHeadModel.from_pretrained("gpt2")

# Training configuration
training_args = TrainingArguments(
    output_dir="./lyrics_generator",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_steps=100,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


model.save_pretrained("./fine_tuned_lyrics_gpt2")
tokenizer.save_pretrained("./fine_tuned_lyrics_gpt2")

Step,Training Loss,Validation Loss


('./fine_tuned_lyrics_gpt2/tokenizer_config.json',
 './fine_tuned_lyrics_gpt2/special_tokens_map.json',
 './fine_tuned_lyrics_gpt2/vocab.json',
 './fine_tuned_lyrics_gpt2/merges.txt',
 './fine_tuned_lyrics_gpt2/added_tokens.json')

In [25]:

from transformers import pipeline

lyrics_generator = pipeline(
    "text-generation",
    model="./fine_tuned_lyrics_gpt2",
    tokenizer=tokenizer
)

print(lyrics_generator("They come from everywhere", max_length=100)[0]["generated_text"])

Device set to use cuda:0


They come from everywhere you go and you're out of touch
I hate you so much I make everybody laugh at you
Cause we're all just trying to kill each other
Yeah, we could be so far apart
We could all get married but we're not into it
I never thought it would happen
I loved when people made my life easier
I loved when I was a kid I thought, yeah, I love you
When you're in pain or when I'm having a
